<a href="https://colab.research.google.com/github/Vethanarrthana/Scientific-Profile-Generator/blob/main/ScientificProfileGenerator_outputParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 5.6 MB/s eta 0:00:00


In [4]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
import os

os.environ["GROQ_API_KEY"]="your api key"

llm=ChatGroq(model="llama3-8b-8192",temperature=0)

In [5]:
response_schemas = [
    ResponseSchema(name="name", description="Name of the scientist"),
    ResponseSchema(name="field", description="Field of the scientist"),
    ResponseSchema(name="notable_work", description="Notable work of the scientist")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    input_variables=["description"],
    template="Describe a scientist and extract details:\n\n{description}\n\n{format_instructions}",
    partial_variables={"format_instructions": format_instructions}
)

description = "A physicist known for the Theory of Relativity"
final_prompt = prompt.format(description=description)

llm_output = llm.invoke(final_prompt)
llm_output_text = llm_output.content
parsed = parser.parse(llm_output_text)

print(parsed)

{'name': 'Albert Einstein', 'field': 'Physicist', 'notable_work': 'Theory of Relativity'}


using pydantic

In [10]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class ScientistProfile(BaseModel):
    name: str = Field(description="Name of the scientist")
    field: str = Field(description="Field of the scientist")
    notable_work: str = Field(description="Notable work of the scientist")

parser = PydanticOutputParser(pydantic_object=ScientistProfile)

prompt = PromptTemplate(
    input_variables=["description"],
    template="Describe a scientist and extract details:\n\n{description}\n\n{format_instructions}",
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

description = input("Please enter a description of a scientist: ")

final_prompt = prompt.format(description=description)
llm_output = llm.invoke(final_prompt)
llm_output_text = llm_output.content

print("Raw LLM output:", llm_output_text)
parsed_data = parser.parse(llm_output_text)

print(parsed_data)

Please enter a description of a scientist: thomas alva edison
Raw LLM output: Here is the output in JSON format that conforms to the given schema:

```
{
  "name": "Thomas Alva Edison",
  "field": "Physicist and Inventor",
  "notable_work": "Development of the Light Bulb"
}
```

Note: The details extracted are based on the information available about Thomas Alva Edison, and may not be exhaustive or entirely accurate.
name='Thomas Alva Edison' field='Physicist and Inventor' notable_work='Development of the Light Bulb'
